In [ ]:
import pandas as pd
from itertools import product

# 1. 读取Excel文件
excel_path = r'C:\Users\huang\work\dataset\calibration\data\123.xlsx'
excel_data = pd.read_excel(excel_path, header=0)

# 2. 提取File字段并替换连字符为下划线
columns = excel_data.columns
file_list = [col.split('_', 3)[-1].replace('-', '_') for col in columns]

# 3. 生成Area字段（行名）
area_list = [f"A1_{i:04d}" for i in range(excel_data.shape[0])]

# 4. 生成笛卡尔积并展开数据
file_area_combinations = list(product(file_list, area_list))  # 笛卡尔积生成所有组合

# 将数据展开为列表
values = excel_data.values.flatten(order='F')  # 按列展开数据（与列名对应）
file_column = [file for file, _ in file_area_combinations]
area_column = [area for _, area in file_area_combinations]

# 5. 创建列表形式的DataFrame
list_format_data = pd.DataFrame({
    "File": file_column,
    "Area": area_column,
    "Dis_cut": values
})

# 6. 保存为CSV文件
output_path = r'C:\Users\huang\work\dataset\calibration\data\123_list_format.csv'
list_format_data.to_csv(output_path, index=False)

print(f"转换完成，结果已保存到 {output_path}")


In [ ]:
import pandas as pd

# 1. 加载两个文件的数据
list_format_path = r'C:\Users\huang\work\dataset\calibration\data\123_list_format.csv'
reference_data_path = r'C:\Users\huang\work\dataset\calibration\data\merged_reference_data.csv'

list_format_data = pd.read_csv(list_format_path)
reference_data = pd.read_csv(reference_data_path)

# 2. 合并两个文件
# 按 "File" 和 "Area" 两个字段匹配，将 list_format_data 的 Dis_cut 合并到 reference_data
merged_data = pd.merge(reference_data, list_format_data[['File', 'Area', 'Dis_cut']], 
                       on=['File', 'Area'], how='left', suffixes=('', '_new'))

# 3. 更新原有的 Dis_cut 列
merged_data['Dis_cut'] = merged_data['Dis_cut_new'].combine_first(merged_data['Dis_cut'])
merged_data.drop(columns=['Dis_cut_new'], inplace=True)  # 删除辅助列

# 4. 保存合并后的结果到新的 CSV 文件
output_path = r'C:\Users\huang\work\dataset\calibration\data\merged_reference_data_updated.csv'
merged_data.to_csv(output_path, index=False)

print(f"合并完成，结果已保存到 {output_path}")